In [ ]:
import sys
import random as random
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Rectangle
import math
random.seed(None)


#Individual takes a step at a random angle from 0 to 2*pi
def rand_angle():
    rand_angle = random.uniform(0, 2*np.pi)
    xdelta = np.cos(rand_angle)
    ydelta = np.sin(rand_angle)
    return xdelta, ydelta

#creates upper and lower corners of a community for the individuals to stay in
def setup_box(box_loc, width, height):
    xloc,yloc = box_loc
    low_left = (xloc - .5 * width, yloc - .5 * height)
    top_right = (xloc + .5 * width, yloc + .5 * height)
    return (low_left, top_right)

#draw the box for the graph
def draw_box(start_loc, boundary):
    xmin, ymin = boundary[0]
    xmax, ymax = boundary[1]
    plt.gca().add_patch(Rectangle((xmin,ymin), xmax-xmin, ymax-ymin, fill = False, edgecolor = 'b', lw = 4))
    plt.xlim([xmin-3,xmax+3])
    plt.ylim([ymin-3,ymax+3])
    plt.axvline(start_loc[0], color='grey', lw=2)
    plt.axhline(start_loc[1], color='grey', lw=2)  
    return

#checks to make sure the individuals remain within the community(travel is not allowed during a lockdown)
def check_inside_box(xvalue, yvalue, boundary):
    xmin, ymin = boundary[0]
    xmax, ymax = boundary[1]
    if xmin < xvalue < xmax and ymin < yvalue < ymax:
        return True
    else:
        return False

def check_infect_others(community_x, community_y, curr_person):
    distances = np.zeros(shape=(num_people,num_people))
    for i in range(num_people):
        for j in range(num_people):
            distance = np.sqrt((community_x[i][-1] - community_x[j][-1])**2 + (community_y[i][-1] - community_y[j][-1])**2 ) 
            distances[i,j] = distance

    threshold = 1
    p_transmission = random.uniform(0.05, 0.63)
    for i in range(num_people):
        for j in range(num_people): 
            if distances[i][j] < threshold and i!=j:
                if infection_status[i] == 1 and infection_status[j] == 0:
                    if random.random() < p_transmission:
                        infection_status[j] = 1
                if infection_status[i] == 0 and infection_status[j] == 1:
                    if random.random() < p_transmission:
                        infection_status[j] = 1

def random_walk(community_x, community_y, infection_status, boundary, boundary_width, boundary_height):
    num_people = len(community_x)
    #starting positions
    for i in range(num_people):
        community_x[i].append(random.uniform(0, boundary_width))
        community_y[i].append(random.uniform(0, boundary_height))

    prior_day = 0
    counter = 0;
    infected_total_days = np.asarray(infection_status)
    while infection_status.count(1) != 0 and infection_status.count(0) != 0:
        for curr_person in range(num_people):
            xdelta, ydelta = rand_angle()
            xtrial = community_x[curr_person][counter] + xdelta
            ytrial = community_y[curr_person][counter] + ydelta
            while check_inside_box(xtrial, ytrial, boundary) != True:
                xdelta, ydelta = rand_angle()
                xtrial = community_x[curr_person][counter] + xdelta
                ytrial = community_y[curr_person][counter] + ydelta
            community_x[curr_person].append(xtrial)
            community_y[curr_person].append(ytrial)
    
            #Draws the whole graph <-- we want to change it to make it draw only the last couple movement
        check_infect_others(community_x, community_y, curr_person)  
          
        days_in_steps = 10
        if counter % days_in_steps == 0:
            #Set plot title , legend, and grid
            for i in range(len(infection_status)):
              infected_total_days[i] += infection_status[i];
              if infected_total_days[i] > 14:
                  if random.uniform(0,1) < 0.02:
                      infection_status[i] = -1
                  elif random.uniform(0,1) > 0.02:
                      infection_status[i] = 2     
            for curr_person in range(num_people):
                if infection_status[curr_person] == 1:
                    plt.plot(community_x[curr_person][counter-days_in_steps::], community_y[curr_person][counter-days_in_steps::], color="red")
                    #,label= f'{curr_person+1}, Day {prior_day}'
                elif infection_status[curr_person] == 0:
                    plt.plot(community_x[curr_person][counter-days_in_steps::], community_y[curr_person][counter-days_in_steps::], color="grey")  
                    #,label= f'{curr_person+1}, Day {prior_day}'  
            plt.suptitle("COVID-19 Transmission Simulator", fontsize=15)

            axes = plt.gca()
            axes.set_xlim([0,boundary_width])
            axes.set_ylim([0,boundary_height])

            plt.title(f'Day {prior_day}, Number of uninfected individuals: {infection_status.count(0)}, Number of infected individuals: {infection_status.count(1)}', fontsize=10)
            #plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
            plt.grid(True)
            plt.show()
            prior_day += 1
        counter += 1
    days_in_steps = 10
    for curr_person in range(num_people):
        if infection_status[curr_person] == 1:
            plt.plot(community_x[curr_person][counter-days_in_steps::], community_y[curr_person][counter-days_in_steps::], color="red")
            #,label= f'{curr_person+1}, Day {prior_day}'
        elif infection_status[curr_person] == 0:
            plt.plot(community_x[curr_person][counter-days_in_steps::], community_y[curr_person][counter-days_in_steps::], color="grey")  
            #,label= f'{curr_person+1}, Day {prior_day}'  
    plt.suptitle("COVID-19 Transmission Simulator", fontsize=15)

    axes = plt.gca()
    axes.set_xlim([0,boundary_width])
    axes.set_ylim([0,boundary_height])

    plt.title(f'Day {prior_day}, Number of uninfected individuals: {infection_status.count(0)}, Number of infected individuals: {infection_status.count(1)}', fontsize=10)
    #plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True)
    plt.show()
    prior_day += 1
    

    recovered = infection_status.count(2)
    infected = infection_status.count(1)
    uninfected = infection_status.count(0) 
    dead = infection_status.count(-1)
    #print(infection_status)
    return recovered, infected, uninfected, dead

for i in range(10):
    #community boundary
    boundary_width = 50;
    boundary_height = 50;
    boundary = setup_box((boundary_width/2, boundary_height/2), boundary_width, boundary_height) 

    #num_people = int(input("Enter the number of people: "))
    num_people = 100

    #population stores all the communities
    community_x = {}
    community_y = {}
    infection_status = [0]*num_people
    infection_status[2] = 1
    #inserting a list for each community
    for i in range(num_people):
        community_x[i] = []
        community_y[i] = []  

    recovered, infected, uninfected, dead = random_walk(community_x, community_y, infection_status, boundary, boundary_width, boundary_height)
    print(f"Recovered {recovered}")
    print(f"Infected {infected}")
    print(f"Uninfected {uninfected}")
    print(f"Dead {dead}")
